In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime


# Inicializa el WebDriver
driver = webdriver.Chrome()

# Maximize the window
driver.maximize_window()

# Get
driver.get("https://www.coinglass.com/es/LiquidationData")

# ZZZ
wait = WebDriverWait(driver, 20)  # Espera hasta 10 segundos

# Lista para almacenar los datos
data = []

# Grupos de XPaths y sus nombres correspondientes
xpath_groups = {
    'TODO': list(range(8, 11)),
    'BINANCE': list(range(14, 17)),
    'OKX': list(range(20, 23)),
    'BYBIT': list(range(26, 29)),
    'HUOBI': list(range(32, 35)),
}


try:
    for group_name, indices in xpath_groups.items():
        
        # Define labels for elements
        element_labels = ["All", "Long", "Short"]
        
        for index, label in zip(indices, element_labels):
            # Construct the XPath
            xpath = f'/html/body/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/div[1]/div/div[2]/div[{index}]/div'
            
            # Find the element and extract the data
            element = driver.find_element(By.XPATH, xpath)
            value = element.get_attribute("aria-label")
            
            # Store the data in the list
            data.append({'Grupo': group_name, 'Elemento': label, 'Valor': value})
        
    # Create a pandas DataFrame
    df = pd.DataFrame(data)

except Exception as e:
    print(f"An error occurred: {e}")


# Cierra el driver
driver.quit()

# Get the current timestamp
current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [13]:
df.head()

,Grupo,Elemento,Valor
0,TODO,All,"22.707.121,465"
1,TODO,Long,"18.390.830,469"
2,TODO,Short,"4.316.290,995"
3,BINANCE,All,"13.630.397,704"
4,BINANCE,Long,"11.928.589,812"


In [14]:
# Convert 'Valor' to a format that can be used for calculations
df['Valor'] = df['Valor'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

In [15]:
# Initialize an empty list to store the new data
new_data = []


# Group by 'Grupo' and iterate over each group to perform calculations
for name, group in df.groupby('Grupo'):
    try:
        all_liquidations = group.loc[group['Elemento'] == 'All', 'Valor'].values[0]
        long_liquidations = group.loc[group['Elemento'] == 'Long', 'Valor'].values[0]
        short_liquidations = group.loc[group['Elemento'] == 'Short', 'Valor'].values[0]
    except IndexError:
        print(f"Data for group {name} is incomplete. Skipping...")
        continue
    
    # Calculate the ratios
    ratio_long_short = long_liquidations / short_liquidations
    ratio_short_long = short_liquidations / long_liquidations
    
    # Append the new data
    new_data.append({
        'Grupo': name, 
        'Total_liquidations/1000': all_liquidations / 1000, 
        'Long/Short Ratio': ratio_long_short, 
        'Short/Long Ratio': ratio_short_long, 
        'Timestamp': current_timestamp
    })

# Create a new DataFrame
new_df = pd.DataFrame(new_data)

# Format the columns
new_df['Total_liquidations/1000'] = new_df['Total_liquidations/1000'].apply(lambda x: "{:,.0f}".format(x))
new_df['Long/Short Ratio'] = new_df['Long/Short Ratio'].apply(lambda x: "{:.2f}".format(x))
new_df['Short/Long Ratio'] = new_df['Short/Long Ratio'].apply(lambda x: "{:.2f}".format(x))

# Reorder the DataFrame to have 'TODO' on top
new_df = new_df.sort_values(by='Grupo', ascending=False)

# Convert 'Total_liquidations/1000' back to float for calculations
new_df['Total_liquidations/1000'] = new_df['Total_liquidations/1000'].str.replace(',', '').astype(float)

# Calculate the Long and Short Liquidations
new_df['Long Liquidations'] = new_df['Total_liquidations/1000'] / (1 + 1/new_df['Long/Short Ratio'].astype(float))
new_df['Short Liquidations'] = new_df['Total_liquidations/1000'] / (1 + new_df['Long/Short Ratio'].astype(float))

# Format the newly calculated columns
new_df['Long Liquidations'] = new_df['Long Liquidations'].apply(lambda x: "{:,.0f}".format(x))
new_df['Short Liquidations'] = new_df['Short Liquidations'].apply(lambda x: "{:,.0f}".format(x))

# Convert all necessary columns to the appropriate data types and format
new_df['Total_liquidations/1000'] = new_df['Total_liquidations/1000'].astype(float)
new_df['Long/Short Ratio'] = new_df['Long/Short Ratio'].astype(float)
new_df['Short/Long Ratio'] = new_df['Short/Long Ratio'].astype(float)

# Monetize and set decimals for 'Total_liquidations/1000', 'Long Liquidations' and 'Short Liquidations' without decimals
new_df['Total_liquidations/1000'] = new_df['Total_liquidations/1000'].apply(lambda x: f"${int(x):,.0f}K")
new_df['Long Liquidations'] = new_df['Long Liquidations'].apply(lambda x: f"${int(float(x.replace('$', '').replace('K', '').replace(',', ''))):,.0f}K")
new_df['Short Liquidations'] = new_df['Short Liquidations'].apply(lambda x: f"${int(float(x.replace('$', '').replace('K', '').replace(',', ''))):,.0f}K")


# Set decimals for 'Long/Short Ratio' and 'Short/Long Ratio'
new_df['Long/Short Ratio'] = new_df['Long/Short Ratio'].apply(lambda x: f"{x:.2f}")
new_df['Short/Long Ratio'] = new_df['Short/Long Ratio'].apply(lambda x: f"{x:.2f}")

# Reorder the columns according to the desired sequence
column_order = ['Timestamp', 'Grupo', 'Total_liquidations/1000', 'Long Liquidations', 'Short Liquidations', 'Long/Short Ratio', 'Short/Long Ratio']
new_df = new_df[column_order]

# Convert 'Total_liquidations/1000' back to float for calculations
new_df['Total_liquidations/1000'] = new_df['Total_liquidations/1000'].apply(lambda x: float(x.replace('$', '').replace('K', '').replace(',', '')))

# Calculate the total liquidations for TODO
total_liquidations_TODO = new_df.loc[new_df['Grupo'] == 'TODO', 'Total_liquidations/1000'].values[0]

# Calculate the '%_Exchanges' column
new_df['%_Exchanges'] = (new_df['Total_liquidations/1000'] / total_liquidations_TODO * 100).apply(lambda x: f"{x:.2f}%")


In [16]:
new_df.head()

,Timestamp,Grupo,Total_liquidations/1000,Long Liquidations,Short Liquidations,Long/Short Ratio,Short/Long Ratio,%_Exchanges
4,2023-10-17 18:26:09,TODO,22707.0,"$18,390K","$4,317K",4.26,0.23,100.00%
3,2023-10-17 18:26:09,OKX,6433.0,"$4,416K","$2,017K",2.19,0.46,28.33%
2,2023-10-17 18:26:09,HUOBI,737.0,$498K,$239K,2.09,0.48,3.25%
1,2023-10-17 18:26:09,BYBIT,1109.0,$768K,$341K,2.25,0.44,4.88%
0,2023-10-17 18:26:09,BINANCE,13630.0,"$11,928K","$1,702K",7.01,0.14,60.03%


In [17]:
# Export to .csv
new_df.to_csv('../processed_data/liquidations24h.csv')

In [18]:
import sqlite3
from contextlib import closing


def read_csv_to_dataframe(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}")
        return None

def create_sqlite_db(dataframe, table_name, conn):
    try:
        dataframe.to_sql(table_name, conn, if_exists='append')
    except Exception as e:
        print(f"An error occurred while creating the SQLite table: {e}")

def query_sqlite_db(query, conn):
    try:
        return pd.read_sql_query(query, conn)
    except Exception as e:
        print(f"An error occurred while querying the SQLite database: {e}")

if __name__ == "__main__":
    csv_file_path = '../processed_data/liquidations24h.csv'
    db_file_path = '../BTC_data.db'
    table_name = 'Liquidations24h'
    
    # Read data from CSV file into DataFrame
    df = read_csv_to_dataframe(csv_file_path)
    if df is not None:
        # Create a SQLite database saved to disk
        with closing(sqlite3.connect(db_file_path)) as conn:
            # Create table and insert data
            create_sqlite_db(df, table_name, conn)
            
            # Query to make sure the data has been inserted properly
            query = f"SELECT * FROM {table_name} LIMIT 5;"
            queried_data = query_sqlite_db(query, conn)
            if queried_data is not None:
                print(queried_data)


   index  Unnamed: 0            Timestamp    Grupo  Total_liquidations/1000  \
0      0           4  2023-10-17 18:18:33     TODO                  22437.0   
1      1           3  2023-10-17 18:18:33      OKX                   6174.0   
2      2           2  2023-10-17 18:18:33    HUOBI                    737.0   
3      3           1  2023-10-17 18:18:33    BYBIT                   1107.0   
4      4           0  2023-10-17 18:18:33  BINANCE                  13603.0   

  Long Liquidations Short Liquidations  Long/Short Ratio  Short/Long Ratio  \
0          $18,473K            $3,964K              4.66              0.21   
1           $4,425K            $1,749K              2.53              0.39   
2             $498K              $239K              2.09              0.48   
3             $768K              $339K              2.27              0.44   
4          $11,980K            $1,623K              7.38              0.14   

  %_Exchanges  
0     100.00%  
1      27.52%  
2       